# Get carbon intensity by country

We get our carbon intensity from [Our World In DAta](https://ourworldindata.org/grapher/carbon-intensity-electricity).

The data source and processing are available on [OWID Github](https://github.com/owid/energy-data).

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/owid/energy-data/master/owid-energy-data.csv")
df

,country,year,iso_code,population,gdp,biofuel_cons_change_pct,biofuel_cons_change_twh,biofuel_cons_per_capita,biofuel_consumption,biofuel_elec_per_capita,...,solar_share_elec,solar_share_energy,wind_cons_change_pct,wind_cons_change_twh,wind_consumption,wind_elec_per_capita,wind_electricity,wind_energy_per_capita,wind_share_elec,wind_share_energy
0,Afghanistan,1900,AFG,4832414.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1901,AFG,4879685.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,1902,AFG,4935122.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,1903,AFG,4998861.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,1904,AFG,5063419.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22338,Zimbabwe,2017,ZWE,14236599.0,2.194784e+10,NaN,NaN,NaN,NaN,22.477,...,0.137,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN
22339,Zimbabwe,2018,ZWE,14438812.0,2.271535e+10,NaN,NaN,NaN,NaN,27.011,...,0.110,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN
22340,Zimbabwe,2019,ZWE,14645473.0,NaN,NaN,NaN,NaN,NaN,25.947,...,0.088,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN
22341,Zimbabwe,2020,ZWE,14862927.0,NaN,NaN,NaN,NaN,NaN,24.221,...,0.090,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN


In [23]:
elec_col = [c for c in df.columns.to_list() if "_electricity" in c]
nuke_col = [c for c in elec_col if "nuclear" in c]
nuke_col

['nuclear_electricity']

In [24]:
carbon_intensity_col = [c for c in df.columns.to_list() if "intensity" in c]

In [25]:
df[['country', 'iso_code', 'year'] + carbon_intensity_col + elec_col].query("iso_code=='FRA' and year==2021")

,country,iso_code,year,carbon_intensity_elec,biofuel_electricity,coal_electricity,fossil_electricity,gas_electricity,hydro_electricity,low_carbon_electricity,nuclear_electricity,oil_electricity,other_renewable_electricity,other_renewable_exc_biofuel_electricity,per_capita_electricity,renewables_electricity,solar_electricity,wind_electricity
7154,France,FRA,2021,58.479,8.67,6.1,50.62,33.92,57.967,500.093,379.361,10.6,11.192,0.61,8363.673,120.732,14.606,36.967


The output we need is a JSON file named `global_energy_mix.json` in this format:
```json
[
    "FRA": {
        "carbon_intensity": 55,
        "country_id": 194,
        "country_name": "France",
        "fossil_TWh": 44.553,
        "geothermal_TWh": 0.155753680282,
        "hydroelectricity_TWh": 60.75027,
        "iso_code": "FRA",
        "nuclear_TWh": 335.41461,
        "official_name_en": "France",
        "region": "Europe",
        "solar_TWh": 14.1455621272,
        "total_TWh": 498.55086677838193,
        "wind_TWh": 43.5316709709,
        "year": 2020
    },
    "USA": {
        "country_id": 48,
        "country_name": "US",
        "fossil_TWh": 2419.23028249,
        "geothermal_TWh": 16.93010517,
        "hydroelectricity_TWh": 291.11084,
        "iso_code": "USA",
        "nuclear_TWh": 789.91864,
        "official_name_en": "United States of America",
        "region": "Americas",
        "solar_TWh": 132.6307535,
        "total_TWh": 3987.3304358499995,
        "wind_TWh": 337.50981469,
        "year": 2020
    },
]
```

We need only the last available indicator for the columns `carbon_intensity_elec`, not the history, `year` is just an indicator of the freshness.

If we don't have `carbon_intensity` it could be computed, but we prefer to not have to do it.

We could keep `*_electricity` to have the electricity mix in `TWh` but it is not mandatory.

So, we have to:
1. Filter for the last available year
1. Check if we not loose country we previously had.
1. Keep only the data we need
1. Export to JSON